In [1]:
import pandas as pd
import numpy as np
import time
import turicreate as tc
from sklearn.cross_validation import train_test_split

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [10]:
userItemData = pd.read_csv('data2.csv',encoding = 'unicode_escape',dtype='str')
#print(userItemData.head())
userList=list(set(userItemData["CustomerID"].tolist()))
#print(len(userList))
#print(userItemData.head(10))
filteredData=pd.DataFrame(columns=('customerId','products'))
rowCount=0
#print(userList) 
for user in userList: 
    products=[] 
    x=userItemData[userItemData.CustomerID==user]['StockCode'].tolist() 
    #print(x)
    s='|' 
    s=s.join(x) 
    filteredData.loc[rowCount]=[user,s] 
    rowCount+=1 
transactions=filteredData.dropna()
print(transactions.head())

  customerId                                           products
1      14176  21733|22645|22804|22810|21355|85123A|22968|211...
2      13305                     85061W|22855|22465|22740|21385
3      14679                                              84678
4      14911  22968|85071A|85071C|22355|21579|21576|22147|22...
5      14673  21485|22835|21143|22158|22633|22419|21212|2217...


In [19]:
customers = pd.DataFrame(transactions['customerId'])
print(type(transactions))

<class 'pandas.core.frame.DataFrame'>


In [13]:
transactions['products'] = transactions['products'].apply(lambda x: [str(i) for i in x.split('|')])
transactions.head(2).set_index('customerId')['products'].apply(pd.Series).reset_index()

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,customerId,0,1,2,3,4,5,6,7,8
0,14176,21733,22645,22804,22810,21355,85123A,22968,21109,21108
1,13305,85061W,22855,22465,22740,21385,NaN,NaN,NaN,NaN


In [20]:
# example 2: organize a given table into a dataframe with customerId, single productId, and purchase count
pd.melt(transactions.head(2).set_index('customerId')['products'].apply(pd.Series).reset_index(), 
             id_vars=['customerId'],
             value_name='products') \
    .dropna().drop(['variable'], axis=1) \
    .groupby(['customerId', 'products']) \
    .agg({'products': 'count'}) \
    .rename(columns={'products': 'purchase_count'}) \
    .reset_index() \
    .rename(columns={'products': 'productId'})

,customerId,productId,purchase_count
0,13305,21385,1
1,13305,22465,1
2,13305,22740,1
3,13305,22855,1
4,13305,85061W,1
5,14176,21108,1
6,14176,21109,1
7,14176,21355,1
8,14176,21733,1
9,14176,22645,1


In [23]:
s=time.time()

data = pd.melt(transactions.set_index('customerId')['products'].apply(pd.Series).reset_index(), 
             id_vars=['customerId'],
             value_name='products') \
    .dropna().drop(['variable'], axis=1) \
    .groupby(['customerId', 'products']) \
    .agg({'products': 'count'}) \
    .rename(columns={'products': 'purchase_count'}) \
    .reset_index() \
    .rename(columns={'products': 'productId'})
data['productId'] = data['productId'].astype(np.str)

print("Execution time:", round((time.time()-s)/60,2), "minutes")

('Execution time:', 0.0, 'minutes')


In [24]:
print(data.shape)
data.head()

(12094, 3)


,customerId,productId,purchase_count
0,12347,20780,1
1,12347,20782,1
2,12347,21064,1
3,12347,21171,1
4,12347,21731,1


In [26]:
def create_data_dummy(data):
    data_dummy = data.copy()
    data_dummy['purchase_dummy'] = 1
    return data_dummy

In [28]:
data_dummy = create_data_dummy(data)

In [29]:
df_matrix = pd.pivot_table(data, values='purchase_count', index='customerId', columns='productId')
df_matrix.head()

productId,10002,10120,10123C,10124A,10124G,10125,10133,10135,11001,15034,...,90214J,90214K,90214M,90214S,90214V,BANK CHARGES,C2,D,M,POST
customerId,,,,,,,,,,,,,,,,,,,,,
12347,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12386,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12395,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
12427,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
12431,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
(df_matrix.shape)

(559, 1969)

In [31]:
df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())
print(df_matrix_norm.shape)
df_matrix_norm.head()

(559, 1969)


productId,10002,10120,10123C,10124A,10124G,10125,10133,10135,11001,15034,...,90214J,90214K,90214M,90214S,90214V,BANK CHARGES,C2,D,M,POST
customerId,,,,,,,,,,,,,,,,,,,,,
12347,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12386,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12395,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
12427,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
12431,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
# create a table for input to the modeling

d = df_matrix_norm.reset_index()
d.index.names = ['scaled_purchase_freq']
data_norm = pd.melt(d, id_vars=['customerId'], value_name='scaled_purchase_freq').dropna()
print(data_norm.shape)
data_norm.head()

(6476, 3)


,customerId,productId,scaled_purchase_freq
3386,12748,10133,1.0
3680,15983,10133,0.0
3801,17259,10133,0.0
3901,18118,10133,0.0
7847,12682,15056BL,0.0


In [34]:
def normalize_data(data):
    df_matrix = pd.pivot_table(data, values='purchase_count', index='customerId', columns='productId')
    df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())
    d = df_matrix_norm.reset_index()
    d.index.names = ['scaled_purchase_freq']
    return pd.melt(d, id_vars=['customerId'], value_name='scaled_purchase_freq').dropna()

In [35]:
train, test = train_test_split(data, test_size = .2)
print(train.shape, test.shape)

((9675, 3), (2419, 3))


In [37]:
# Using turicreate library, we convert dataframe to SFrame - this will be useful in the modeling part

train_data = tc.SFrame(train)
test_data = tc.SFrame(test)

In [38]:
train_data

customerId,productId,purchase_count
12921,21218,1
17402,22069,1
15880,84744,1
13174,84531A,2
16221,21034,1
18118,21207,1
13838,22470,1
16931,20967,1
13402,22624,1
17677,22961,1


In [39]:
test_data

customerId,productId,purchase_count
16081,21531,1
17228,20975,1
17287,22988,2
18043,84910A,1
17377,82494L,1
15860,22449,1
14766,21485,1
17581,72801D,1
15356,20978,1
13089,21481,1


In [41]:
# We can define a function for this step as follows

def split_data(data):
    '''
    Splits dataset into training and test set.
    
    Args:
        data (pandas.DataFrame)
        
    Returns
        train_data (tc.SFrame)
        test_data (tc.SFrame)
    '''
    train, test = train_test_split(data, test_size = .2)
    train_data = tc.SFrame(train)
    test_data = tc.SFrame(test)
    return train_data, test_data

In [42]:
# lets try with both dummy table and scaled/normalized purchase table

train_data_dummy, test_data_dummy = split_data(data_dummy)
train_data_norm, test_data_norm = split_data(data_norm)

In [43]:
# variables to define field names
user_id = 'customerId'
item_id = 'productId'
target = 'purchase_count'
users_to_recommend = list(transactions[user_id])
n_rec = 10 # number of items to recommend
n_display = 30

In [44]:
popularity_model = tc.popularity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target)

Preparing data set.

Data has 9675 observations with 551 users and 1820 items.

Data prepared in: 0.031525s

9675 observations to process; with 1820 unique items.

In [45]:
# Get recommendations for a list of users to recommend (from customers file)
# Printed below is head / top 30 rows for first 3 customers with 10 recommendations each

popularity_recomm = popularity_model.recommend(users=users_to_recommend, k=n_rec)
popularity_recomm.print_rows(n_display)

+------------+-----------+---------------+------+
| customerId | productId |     score     | rank |
+------------+-----------+---------------+------+
|   14176    |   22803   |      8.0      |  1   |
|   14176    |   21448   |      5.0      |  2   |
|   14176    |   21071   |      3.8      |  3   |
|   14176    |   71053   | 3.66666666667 |  4   |
|   14176    |   90199C  |      3.0      |  5   |
|   14176    |   90082B  |      3.0      |  6   |
|   14176    |   21415   |      3.0      |  7   |
|   14176    |   21785   |      3.0      |  8   |
|   14176    |   37370   |      2.75     |  9   |
|   14176    |   84827   |      2.5      |  10  |
|   13305    |   22803   |      8.0      |  1   |
|   13305    |   21448   |      5.0      |  2   |
|   13305    |   21071   |      3.8      |  3   |
|   13305    |   71053   | 3.66666666667 |  4   |
|   13305    |   90199C  |      3.0      |  5   |
|   13305    |   90082B  |      3.0      |  6   |
|   13305    |   21415   |      3.0      |  7   |


In [47]:
# Since turicreate is very accessible library, we can define a model selection function as below

def model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display):
    if name == 'popularity':
        model = tc.popularity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target)
    elif name == 'cosine':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='cosine')
    elif name == 'pearson':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='pearson')
        
    recom = model.recommend(users=users_to_recommend, k=n_rec)
    recom.print_rows(n_display)
    return model

In [49]:
# variables to define field names
# constant variables include:
user_id = 'customerId'
item_id = 'productId'
users_to_recommend = list(customers[user_id])
n_rec = 10 # number of items to recommend
n_display = 30 # to print the head / first few rows in a defined dataset

In [50]:
# these variables will change accordingly
name = 'popularity'
target = 'purchase_dummy'
pop_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 9675 observations with 548 users and 1863 items.

Data prepared in: 0.068624s

9675 observations to process; with 1863 unique items.

+------------+-----------+-------+------+
| customerId | productId | score | rank |
+------------+-----------+-------+------+
|   14176    |   85099F  |  1.0  |  1   |
|   14176    |   21743   |  1.0  |  2   |
|   14176    |   22114   |  1.0  |  3   |
|   14176    |   21488   |  1.0  |  4   |
|   14176    |   82494L  |  1.0  |  5   |
|   14176    |   22592   |  1.0  |  6   |
|   14176    |   84031A  |  1.0  |  7   |
|   14176    |   82583   |  1.0  |  8   |
|   14176    |   21125   |  1.0  |  9   |
|   14176    |   22569   |  1.0  |  10  |
|   13305    |   85099F  |  1.0  |  1   |
|   13305    |   21743   |  1.0  |  2   |
|   13305    |   22114   |  1.0  |  3   |
|   13305    |   21488   |  1.0  |  4   |
|   13305    |   82494L  |  1.0  |  5   |
|   13305    |   22592   |  1.0  |  6   |
|   13305    |   84031A  |  1.0  |  7   |
|   13305    |   82583   |  1.0  |  8   |
|   13305    |   21125   |  1.0  |  9   |
|   13305    |   22569   |  1.0  |  10  |
|   14679    |   85099F  |  1.0  |

In [51]:
name = 'popularity'
target = 'scaled_purchase_freq'
pop_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 5180 observations with 519 users and 500 items.

Data prepared in: 0.057747s

5180 observations to process; with 500 unique items.

+------------+-----------+-------+------+
| customerId | productId | score | rank |
+------------+-----------+-------+------+
|   14176    |   21822   |  1.0  |  1   |
|   14176    |   90082B  |  1.0  |  2   |
|   14176    |   35971   |  1.0  |  3   |
|   14176    |   22547   |  1.0  |  4   |
|   14176    |   21520   |  1.0  |  5   |
|   14176    |   20780   |  1.0  |  6   |
|   14176    |   72131   |  1.0  |  7   |
|   14176    |   85232B  |  1.0  |  8   |
|   14176    |   21383   |  1.0  |  9   |
|   14176    |   22803   |  1.0  |  10  |
|   13305    |   21822   |  1.0  |  1   |
|   13305    |   90082B  |  1.0  |  2   |
|   13305    |   35971   |  1.0  |  3   |
|   13305    |   22547   |  1.0  |  4   |
|   13305    |   21520   |  1.0  |  5   |
|   13305    |   20780   |  1.0  |  6   |
|   13305    |   72131   |  1.0  |  7   |
|   13305    |   85232B  |  1.0  |  8   |
|   13305    |   21383   |  1.0  |  9   |
|   13305    |   22803   |  1.0  |  10  |
|   14679    |   21822   |  1.0  |

In [52]:
train.groupby(by=item_id)['purchase_count'].mean().sort_values(ascending=False).head(20)

productId
22803     8.000000
21448     5.000000
21071     3.800000
71053     3.666667
21785     3.000000
90082B    3.000000
90199C    3.000000
21415     3.000000
37370     2.750000
84827     2.500000
47420     2.000000
35833P    2.000000
21696     2.000000
21392     2.000000
72817     2.000000
84685     2.000000
51014L    2.000000
21202     2.000000
84658     2.000000
21647     2.000000
Name: purchase_count, dtype: float64

In [53]:
# these variables will change accordingly
name = 'cosine'
target = 'purchase_count'
cos = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 9675 observations with 551 users and 1820 items.

Data prepared in: 0.021888s

Training model from provided data.

Gathering per-item and per-user statistics.

...

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 1.11537s

+------------+-----------+-----------------+------+
| customerId | productId |      score      | rank |
+------------+-----------+-----------------+------+
|   14176    |   20679   |  0.228090337345 |  1   |
|   14176    |  15056BL  |  0.197930599962 |  2   |
|   14176    |   71053   | 0.0807081716401 |  3   |
|   14176    |   22589   | 0.0792487433978 |  4   |
|   14176    |   21812   | 0.0738944666726 |  5   |
|   14176    |   21733   | 0.0728338445936 |  6   |
|   14176    |   84925D  | 0.0728281310626 |  7   |
|   14176    |   22137   | 0.0692958916937 |  8   |
|   14176    |   22644   | 0.0690705435617 |  9   |
|   14176    |   84029E  | 0.0687695486205 |  10  |
|   13305    |   21640   |  0.192450086276 |  1   |
|   13305    |   22797   |  0.192450086276 |  2   |
|   13305    |   21906   |  0.145478586356 |  3   |
|   13305    |   22116   |  0.13608276844  |  4   |
|   13305    |   21894   |  0.13608276844  |  5   |
|   13305    |   72351B  |  0.13608276844  |  6   |
|   13305   

In [54]:
# these variables will change accordingly
name = 'cosine'
target = 'scaled_purchase_freq'
cos_norname = 'cosine'
target = 'purchase_dummy'
cos_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 9675 observations with 548 users and 1863 items.

Data prepared in: 0.041176s

...

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.069773s

+------------+-----------+-----------------+------+
| customerId | productId |      score      | rank |
+------------+-----------+-----------------+------+
|   14176    |   22513   | 0.0916864540842 |  1   |
|   14176    |   71496A  | 0.0916864540842 |  2   |
|   14176    |   78124   | 0.0916864540842 |  3   |
|   14176    |   71495B  | 0.0916864540842 |  4   |
|   14176    |   22483   | 0.0916864540842 |  5   |
|   14176    |   22359   | 0.0875125792291 |  6   |
|   14176    |   21201   | 0.0875125792291 |  7   |
|   14176    |   21205   | 0.0817654132843 |  8   |
|   14176    |   21376   | 0.0817654132843 |  9   |
|   14176    |   21896   | 0.0817654132843 |  10  |
|   13305    |   22251   |      0.125      |  1   |
|   13305    |   85129C  |      0.125      |  2   |
|   13305    |   21640   |      0.125      |  3   |
|   13305    |   22116   |      0.125      |  4   |
|   13305    |   84519B  |  0.111803397536 |  5   |
|   13305    |   22610   |  0.111803397536 |  6   |
|   13305   

In [ ]:
m = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

In [56]:
# these variables will change accordingly
name = 'pearson'
target = 'purchase_count'
pear = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 9675 observations with 551 users and 1820 items.

Data prepared in: 0.023101s

Training model from provided data.

Gathering per-item and per-user statistics.

...

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.0405s

+------------+-----------+---------------+------+
| customerId | productId |     score     | rank |
+------------+-----------+---------------+------+
|   14176    |   22803   |      8.0      |  1   |
|   14176    |   21448   |      5.0      |  2   |
|   14176    |   21071   | 3.76402133363 |  3   |
|   14176    |   71053   | 3.62950806675 |  4   |
|   14176    |   90199C  |      3.0      |  5   |
|   14176    |   90082B  |      3.0      |  6   |
|   14176    |   21415   |      3.0      |  7   |
|   14176    |   21785   |      3.0      |  8   |
|   14176    |   37370   | 2.71129606451 |  9   |
|   14176    |   84827   |      2.5      |  10  |
|   13305    |   22803   |      8.0      |  1   |
|   13305    |   21448   |      5.0      |  2   |
|   13305    |   21071   | 3.69013918519 |  3   |
|   13305    |   71053   | 3.55229116003 |  4   |
|   13305    |   90199C  |      3.0      |  5   |
|   13305    |   90082B  |      3.0      |  6   |
|   13305    |   21415   |      3.0      |  7   |


In [57]:
# these variables will change accordingly
name = 'pearson'
target = 'purchase_dummy'
pear_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 9675 observations with 548 users and 1863 items.

Data prepared in: 0.022253s

...

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.040489s

+------------+-----------+-------+------+
| customerId | productId | score | rank |
+------------+-----------+-------+------+
|   14176    |   85099F  |  0.0  |  1   |
|   14176    |   21743   |  0.0  |  2   |
|   14176    |   22114   |  0.0  |  3   |
|   14176    |   21488   |  0.0  |  4   |
|   14176    |   82494L  |  0.0  |  5   |
|   14176    |   22592   |  0.0  |  6   |
|   14176    |   84031A  |  0.0  |  7   |
|   14176    |   82583   |  0.0  |  8   |
|   14176    |   21125   |  0.0  |  9   |
|   14176    |   22569   |  0.0  |  10  |
|   13305    |   85099F  |  0.0  |  1   |
|   13305    |   21743   |  0.0  |  2   |
|   13305    |   22114   |  0.0  |  3   |
|   13305    |   21488   |  0.0  |  4   |
|   13305    |   82494L  |  0.0  |  5   |
|   13305    |   22592   |  0.0  |  6   |
|   13305    |   84031A  |  0.0  |  7   |
|   13305    |   82583   |  0.0  |  8   |
|   13305    |   21125   |  0.0  |  9   |
|   13305    |   22569   |  0.0  |  10  |
|   14679    |   85099F  |  0.0  |

In [ ]:
name = 'pearson'
target = 'scaled_purchase_freq'
pear_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

In [59]:
# create initial callable variables

models_w_counts = [popularity_model, cos, pear]
models_w_dummy = [pop_dummy, cos_dummy, pear_dummy]
models_w_norm = [pop_norm, cos_norm, pear_norm]

names_w_counts = ['Popularity Model on Purchase Counts', 'Cosine Similarity on Purchase Counts', 'Pearson Similarity on Purchase Counts']
names_w_dummy = ['Popularity Model on Purchase Dummy', 'Cosine Similarity on Purchase Dummy', 'Pearson Similarity on Purchase Dummy']
names_w_norm = ['Popularity Model on Scaled Purchase Counts', 'Cosine Similarity on Scaled Purchase Counts', 'Pearson Similarity on Scaled Purchase Counts']

In [ ]:
eval_counts = tc.recommender.util.compare_models(test_data, models_w_counts, model_names=names_w_counts)

In [61]:
eval_dummy = tc.recommender.util.compare_models(test_data_dummy, models_w_dummy, model_names=names_w_dummy)

PROGRESS: Evaluate model Popularity Model on Purchase Dummy

Precision and recall summary statistics by cutoff
+--------+------------------+------------------+
| cutoff |  mean_precision  |   mean_recall    |
+--------+------------------+------------------+
|   1    |       0.0        |       0.0        |
|   2    |       0.0        |       0.0        |
|   3    | 0.00070771408351 | 0.00212314225053 |
|   4    | 0.0015923566879  | 0.00273256271133 |
|   5    | 0.00169851380042 | 0.00290949123221 |
|   6    | 0.00176928520878 | 0.00361720531572 |
|   7    | 0.00151653017895 | 0.00361720531572 |
|   8    | 0.00132696390658 | 0.00361720531572 |
|   9    | 0.00117952347252 | 0.00361720531572 |
|   10   | 0.00106157112527 | 0.00361720531572 |
+--------+------------------+------------------+
[10 rows x 3 columns]


Overall RMSE: 0.0

Per User RMSE (best)
+------------+-------+------+
| customerId | count | rmse |
+------------+-------+------+
|   15983    |   11  | 0.0  |
+------------+-----

In [62]:
eval_norm = tc.recommender.util.compare_models(test_data_norm, models_w_norm, model_names=names_w_norm)

PROGRESS: Evaluate model Popularity Model on Scaled Purchase Counts

Precision and recall summary statistics by cutoff
+--------+-------------------+-------------------+
| cutoff |   mean_precision  |    mean_recall    |
+--------+-------------------+-------------------+
|   1    |        0.0        |        0.0        |
|   2    |  0.00127877237852 | 0.000319693094629 |
|   3    | 0.000852514919011 | 0.000319693094629 |
|   4    |  0.00127877237852 | 0.000831202046036 |
|   5    |  0.00153452685422 |  0.00108695652174 |
|   6    |  0.00255754475703 |  0.00266410912191 |
|   7    |  0.00255754475703 |  0.00394288150043 |
|   8    |  0.0022378516624  |  0.00394288150043 |
|   9    |  0.00255754475703 |  0.00483802216539 |
|   10   |  0.00281329923274 |  0.0055200341006  |
+--------+-------------------+-------------------+
[10 rows x 3 columns]


Overall RMSE: 0.345690733934

Per User RMSE (best)
+------------+-------+------+
| customerId | count | rmse |
+------------+-------+------+
| 

In [63]:
users_to_recommend = list(customers[user_id])

final_model = tc.item_similarity_recommender.create(tc.SFrame(data_dummy), 
                                            user_id=user_id, 
                                            item_id=item_id, 
                                            target='purchase_dummy', 
                                            similarity_type='cosine')

recom = final_model.recommend(users=users_to_recommend, k=n_rec)
recom.print_rows(n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 12094 observations with 559 users and 1969 items.

Data prepared in: 0.02361s

...

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.074005s

+------------+-----------+-----------------+------+
| customerId | productId |      score      | rank |
+------------+-----------+-----------------+------+
|   14176    |   84709B  | 0.0828823844592 |  1   |
|   14176    |   22068   | 0.0828823844592 |  2   |
|   14176    |   78124   | 0.0812797678841 |  3   |
|   14176    |   71496A  | 0.0812797678841 |  4   |
|   14176    |   71495B  | 0.0812797678841 |  5   |
|   14176    |   22513   | 0.0812797678841 |  6   |
|   14176    |   21201   | 0.0792641573482 |  7   |
|   14176    |   84509A  | 0.0746109551854 |  8   |
|   14176    |   21205   | 0.0728127360344 |  9   |
|   14176    |   20618   | 0.0728127360344 |  10  |
|   13305    |   22116   |  0.170710682869 |  1   |
|   13305    |   85129C  |  0.170710682869 |  2   |
|   13305    |   85130A  |  0.139384686947 |  3   |
|   13305    |   21333   |  0.136251199245 |  4   |
|   13305    |   21179   |  0.120710682869 |  5   |
|   13305    |   21640   |       0.1       |  6   |
|   13305   

In [64]:
df_rec = recom.to_dataframe()
print(df_rec.shape)
df_rec.head()

(5590, 4)


,customerId,productId,score,rank
0,14176,84709B,0.082882,1
1,14176,22068,0.082882,2
2,14176,78124,0.081280,3
3,14176,71496A,0.081280,4
4,14176,71495B,0.081280,5


In [68]:
df_rec['recommendedProducts'] = df_rec.groupby([user_id])[item_id].transform(lambda x: '|'.join(x.astype(str)))
df_output = df_rec[['customerId', 'recommendedProducts']].drop_duplicates().sort_values('customerId').set_index('customerId')
df_output.head()

,recommendedProducts
customerId,
12347,22730|84952A|21846|84997A|20655|21672|85068|47...
12386,22515|21534|22551|84658|84247E|84534B|22553|20...
12395,84991|22951|85170D|22134|22949|22695|21213|229...
12427,21327|22831|22703|22026|21650|21209|20839|2077...
12431,22728|22729|22725|22730|20655|15056N|47563A|21...


In [72]:
 df_output.to_csv('output/option1_recommendation.csv')
print("An output file can be found in 'output' folder with name 'option1_recommendation.csv'")


An output file can be found in 'output' folder with name 'option1_recommendation.csv'


In [ ]:
df_output = create_output(pear_norm, users_to_recommend, n_rec, print_csv=True)
print(df_output.shape)
df_output.head()